In [1]:
import os
import shutil
import tempfile
import time
import matplotlib.pyplot as plt
from monai.losses import DiceLoss
from monai.utils import set_determinism
import onnxruntime
from tqdm import tqdm
import random
from monai.transforms import (
    Activations,
    Activationsd,
    AsDiscrete,
    AsDiscreted,
    Compose,
    Invertd,
    LoadImaged,
    MapTransform,
    NormalizeIntensityd,
    Orientationd,
    RandFlipd,
    RandScaleIntensityd,
    RandShiftIntensityd,
    RandSpatialCropd,
    Spacingd,
    EnsureTyped,
    ResizeWithPadOrCropd,
    EnsureChannelFirstd,
    RandZoomd
)
from monai.data import DataLoader, CacheDataset
import numpy as np
import torch.nn.functional as F
from pathlib import Path


if 'MASTER_ADDR' not in os.environ:
    os.environ['MASTER_ADDR'] = '127.0.0.1'
if 'MASTER_PORT' not in os.environ:
    os.environ['MASTER_PORT'] = '29500'

import torch
import time
from model_builder import UNet3D
from tomultichannel import ConvertToMultiChannel
from preprocess import preprocess_data, show_image
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
import torch.multiprocessing as mp
from glob import glob
import nibabel as nib
from RepeatChannel import RepeatChannelsd
# from monai.data import NiftiSaver
import numpy as np
from U_Mamba_net import U_Mamba_net
from monai.transforms import AsDiscrete, Activations, Compose, Resize   # keep
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_determinism(seed=0)

NUM_WORKERS = 1
# MODEL_PATH = r"model/Medical_Image_UNet3D.pth"
# MODEL_PATH = r"/home/luudh/luudh/MyFile/medical_image_lab/monai/going_modular/model/Medical_Image_U_Mamba_Net_ssm_16_3D.pth"
MODEL_PATH = r"/home/luudh/luudh/MyFile/medical_image_lab/monai/going_modular/model/Medical_Image_U_Mamba_Net_ssm_8_3D_add_AUX_W2_W3_pos_16.pth"
BASE_DIR_LINUX = r"/home/luudh/luudh/MyFile/medical_image_lab/monai/data/test_data/"
# BASE_DIR_LINUX = r"/home/luudh/luudh/MyFile/medical_image_lab/monai/data/Task01_BrainTumour/imagesVal"
torch.cuda.empty_cache()
# Use the same ROI as training to avoid shape drift
TRAIN_ROI = (96, 96, 96)

